<a href="https://colab.research.google.com/github/papago2355/AI-capstone/blob/main/cnn_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [3]:
inputs = pd.read_csv('drive/MyDrive/content/train.csv')
labels = pd.read_csv('drive/MyDrive/content/train_label.csv')
test = pd.read_csv('drive/MyDrive/content/test.csv')

In [4]:
time_series_length = pd.concat([inputs['EMAIL'].value_counts()])
shortest_length = time_series_length[-1]

In [5]:
arranged_labels = []
for id in inputs['EMAIL'].unique():
            idx = inputs['EMAIL'][inputs['EMAIL'] == id].index
            start_idx = idx[0]
            end_idx = idx[-1]
            inputs.drop(list((range(start_idx + shortest_length , end_idx+1))), axis=0, inplace=True)
            inputs = inputs.reset_index(drop=True)
            label_idx = labels['SAMPLE_EMAIL'][labels['SAMPLE_EMAIL'] == id].index[0]
            arranged_labels.append(labels['DIAG_NM'][label_idx])

In [6]:
del_col = [ 'EMAIL', 'summary_date',
                   'activity_class_5min', 'activity_met_1min',
                   'sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min', 'timezone', 'sleep_total',
                   'CONVERT(activity_class_5min USING utf8)', 'CONVERT(activity_met_1min USING utf8)',
                   'CONVERT(sleep_hr_5min USING utf8)', 'CONVERT(sleep_hypnogram_5min USING utf8)',
                   'CONVERT(sleep_rmssd_5min USING utf8)']
inputs.drop(del_col, axis=1, inplace=True)
test.drop(del_col, axis=1, inplace=True)

In [7]:
inputs = inputs.drop(columns = ['activity_average_met', 'activity_cal_active', 'activity_cal_total','activity_steps', 'activity_score',
                                    'activity_score_move_every_hour', 'activity_daily_movement','activity_met_min_high',
                                    'activity_score_stay_active', 'activity_total', 'activity_met_min_low','activity_medium',
                                    'activity_score_training_frequency','activity_steps', 'activity_score',
                                    'activity_score_training_volume','sleep_efficiency','sleep_duration','sleep_hr_lowest',
                                    'sleep_midpoint_time','sleep_awake','sleep_deep','sleep_rem','sleep_restless','sleep_temperature_delta',
                                    'sleep_score_total','sleep_score','activity_score_meet_daily_targets','sleep_period_id','activity_inactivity_alerts','activity_score_recovery_time'])
test = test.drop(columns = ['activity_average_met', 'activity_cal_active', 'activity_cal_total','activity_steps', 'activity_score',
                                    'activity_score_move_every_hour', 'activity_daily_movement','activity_met_min_high',
                                    'activity_score_stay_active', 'activity_total', 'activity_met_min_low','activity_medium',
                                    'activity_score_training_frequency','activity_steps', 'activity_score',
                                    'activity_score_training_volume','sleep_efficiency','sleep_duration','sleep_hr_lowest',
                                    'sleep_midpoint_time','sleep_awake','sleep_deep','sleep_rem','sleep_restless','sleep_temperature_delta',
                                    'sleep_score_total','sleep_score','activity_score_meet_daily_targets','sleep_period_id','activity_inactivity_alerts','activity_score_recovery_time'])

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(inputs)
inputs = scaler.transform(inputs)

In [9]:
arranged_labels = np.reshape(arranged_labels,(-1,1))

In [10]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = OneHotEncoder()
le.fit(arranged_labels)
arranged_labels = le.fit_transform(arranged_labels).toarray()

In [11]:
le.categories_

[array(['CN', 'Dem', 'MCI'], dtype='<U3')]

In [12]:
# cnn lstm model


x = inputs.reshape((148,35,22))

In [13]:
y = arranged_labels

In [14]:
from sklearn.utils.class_weight import compute_class_weight

y_integers = np.argmax(y, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))

In [15]:
d_class_weights

{0: 0.5085910652920962, 1: 4.933333333333334, 2: 1.2032520325203253}

In [16]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout, Flatten, BatchNormalization, Input, Convolution2D, Activation,TimeDistributed
from keras.layers import Input, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import GlobalAveragePooling1D, Permute, Dropout
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

In [17]:
def generate_model():
    ip = Input(shape=(35, 22))

    x = Masking()(ip)
    x = LSTM(units = 50, return_sequences = True)(x)
    x = LSTM(units=50)(x)
    x = Dropout(0.5)(x)

  
    y = Reshape((770,1))(ip)
    y = Conv1D (kernel_size=3, filters=64, strides=3, padding='valid',kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    
    y = Conv1D (kernel_size=3, filters=128, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y)

    y = Conv1D (kernel_size=3, filters=64, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y)

    
    y = Dropout(0.5)(y)
    y = Flatten()(y)
    
    x = concatenate([x, y])

    out = Dense(3, activation='softmax')(x)

    model = Model(ip, out)
    #model.summary()
    return model
model = generate_model()
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 35, 22)]     0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 770, 1)       0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 256, 64)      256         reshape[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 64)      256         conv1d[0][0]                     
______________________________________________________________________________________________

In [18]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(max_train_size=None, n_splits=5)


for train_index, test_index in tscv.split(x):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27] TEST: [28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51] TEST: [52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75] TEST: [76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82

In [19]:
temp = le.inverse_transform(y_train).ravel()
stack = 0
for i in temp:
  if i =='Dem':
    stack+=1
print(stack)

8


In [20]:
X_train.shape

(124, 35, 22)

In [21]:
#early stopping not working!
"""
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=10, mode='auto')
num_models=25
model_list=[]


for i in tqdm(range(num_models)):
    model = generate_model()
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    model.fit(x,y, batch_size=16,validation_data=(X_test,y_test),callbacks=[early_stopping],epochs=100 )
    model_list.append(model)
    model.save(f"model_{i}.h5")
"""

'\nfrom keras.optimizers import SGD\nfrom keras.callbacks import EarlyStopping\nearly_stopping = EarlyStopping(patience=10, mode=\'auto\')\nnum_models=25\nmodel_list=[]\n\n\nfor i in tqdm(range(num_models)):\n    model = generate_model()\n    model.compile(loss=\'categorical_crossentropy\',\n              optimizer=\'adam\',\n              metrics=[\'accuracy\'])\n    model.fit(x,y, batch_size=16,validation_data=(X_test,y_test),callbacks=[early_stopping],epochs=100 )\n    model_list.append(model)\n    model.save(f"model_{i}.h5")\n'

In [22]:
#I already have an enough models...
"""
from matplotlib import pyplot
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=7, mode='auto')
for i in range(0,25):
  model = generate_model()
  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  history = model.fit(x, y, epochs=100, validation_data=(X_test, y_test),callbacks=[early_stopping],class_weight=d_class_weights)
  model.save(f"drive/MyDrive/content/model_{i}.h5")
  pyplot.plot(history.history['loss'])
  pyplot.plot(history.history['val_loss'])
  pyplot.title('model train vs validation loss')
  pyplot.ylabel('loss')
  pyplot.xlabel('epoch')
  pyplot.legend(['train', 'validation'], loc='upper right')
  pyplot.show()
  """

'\nfrom matplotlib import pyplot\nfrom keras.callbacks import EarlyStopping\nearly_stopping = EarlyStopping(patience=7, mode=\'auto\')\nfor i in range(0,25):\n  model = generate_model()\n  model.compile(loss=\'categorical_crossentropy\',\n                optimizer=\'adam\',\n                metrics=[\'accuracy\'])\n  history = model.fit(x, y, epochs=100, validation_data=(X_test, y_test),callbacks=[early_stopping],class_weight=d_class_weights)\n  model.save(f"drive/MyDrive/content/model_{i}.h5")\n  pyplot.plot(history.history[\'loss\'])\n  pyplot.plot(history.history[\'val_loss\'])\n  pyplot.title(\'model train vs validation loss\')\n  pyplot.ylabel(\'loss\')\n  pyplot.xlabel(\'epoch\')\n  pyplot.legend([\'train\', \'validation\'], loc=\'upper right\')\n  pyplot.show()\n  '

In [23]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=20, mode='auto')
#epoch 23 works best
model = generate_model()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x, y, epochs=500, validation_data=(X_test, y_test),callbacks=[early_stopping],class_weight=d_class_weights)
#model.fit(x, y, epochs=23, validation_data=(X_test, y_test),class_weight=d_class_weights)
model.save("drive/MyDrive/content/model_original.h5")

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/500
5/5 [==============================] - 29s 689ms/step - loss: 2.0793 - accuracy: 0.3511 - val_loss: 0.9294 - val_accuracy: 0.6667
Epoch 2/500
5/5 [==============================] - 0s 22ms/step - loss: 1.1722 - accuracy: 0.4466 - val_loss: 3.2515 - val_accuracy: 0.1667
Epoch 3/500
5/5 [==============================] - 0s 22ms/step - loss: 0.9122 - accuracy: 0.5667 - val_loss: 3.2766 - val_accuracy: 0.0833
Epoch 4/500
5/5 [==============================] - 0s 21ms/step - loss: 0.8388 - accuracy: 0.6131 - val_loss: 2.8790 - val_accuracy: 0.1250
Epoch 5/500
5/5 [==============================] - 0s 20ms/step - loss: 0.5600 - accuracy: 0.6443 - val_loss: 1.8856 - val_accuracy: 0.2083
Epoch 6/500
5/5 [==============================] - 0s 21ms/step - loss: 0.4794 - accuracy: 0.7639 - val_loss: 1.5340 - val_accuracy: 0.2500
Epoch 7/500
5/5 [===

In [24]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

1/1 [==============================] - 0s 19ms/step - loss: 3.3674e-04 - accuracy: 1.0000
Test accuracy 1.0
Test loss 0.00033673885627649724


In [25]:
yhat = model.predict(x)
train_label = []
for i in range(0,148):
    if yhat[i][0] > yhat[i][1] and yhat[i][0] > yhat[i][2]:
        train_label.append('CN')
    elif yhat[i][1] > yhat[i][0] and yhat[i][1] > yhat[i][2]:
        train_label.append('Dem')
    elif yhat[i][2] > yhat[i][1] and yhat[i][2] > yhat[i][0]:
        train_label.append('MCI')

In [26]:
arranged_labels = le.inverse_transform(arranged_labels).ravel()

In [27]:
from sklearn.metrics import f1_score
f1_score(arranged_labels, train_label, average='macro')

1.0

In [28]:
test = pd.read_csv('drive/MyDrive/content/test.csv')

In [29]:
time_series_length = pd.concat([test['EMAIL'].value_counts()])
shortest_length = time_series_length[-1]

In [30]:
shortest_length

35

In [31]:
test.shape

(29092, 65)

In [32]:
for id in test['EMAIL'].unique():
            idx = test['EMAIL'][test['EMAIL'] == id].index
            start_idx = idx[0]
            end_idx = idx[-1]
            test.drop(list((range(start_idx + shortest_length , end_idx+1))), axis=0, inplace=True)
            test = test.reset_index(drop=True)

In [33]:
test.shape

(16485, 65)

In [34]:
del_col = [ 'EMAIL', 'summary_date',
                   'activity_class_5min', 'activity_met_1min',
                   'sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min', 'timezone', 'sleep_total',
                   'CONVERT(activity_class_5min USING utf8)', 'CONVERT(activity_met_1min USING utf8)',
                   'CONVERT(sleep_hr_5min USING utf8)', 'CONVERT(sleep_hypnogram_5min USING utf8)',
                   'CONVERT(sleep_rmssd_5min USING utf8)']

test.drop(del_col, axis=1, inplace=True)

In [35]:
test = test.drop(columns = ['activity_average_met', 'activity_cal_active', 'activity_cal_total','activity_steps', 'activity_score',
                                    'activity_score_move_every_hour', 'activity_daily_movement','activity_met_min_high',
                                    'activity_score_stay_active', 'activity_total', 'activity_met_min_low','activity_medium',
                                    'activity_score_training_frequency','activity_steps', 'activity_score',
                                    'activity_score_training_volume','sleep_efficiency','sleep_duration','sleep_hr_lowest',
                                    'sleep_midpoint_time','sleep_awake','sleep_deep','sleep_rem','sleep_restless','sleep_temperature_delta',
                                    'sleep_score_total','sleep_score','activity_score_meet_daily_targets','sleep_period_id','activity_inactivity_alerts','activity_score_recovery_time'])

In [36]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(test)
test = scaler.transform(test)

In [37]:
test.shape

(16485, 22)

In [38]:
x_test = test.reshape((471,35,22))

In [39]:
yhat = model.predict(x_test)

In [40]:
yhat

array([[9.9923480e-01, 6.6231260e-05, 6.9894904e-04],
       [9.9805570e-01, 2.7420092e-04, 1.6700751e-03],
       [9.9978226e-01, 1.9995385e-04, 1.7750366e-05],
       ...,
       [5.3710955e-01, 1.3499437e-03, 4.6154052e-01],
       [9.3975085e-01, 1.8189403e-03, 5.8430139e-02],
       [9.9607605e-01, 5.6398268e-05, 3.8675743e-03]], dtype=float32)

In [41]:
test_labels = pd.read_csv('drive/MyDrive/content/sample_submission.csv')

In [42]:
for i in range(0,471):
    if yhat[i][0] > yhat[i][1] and yhat[i][0] > yhat[i][2]:
        test_labels['DIAG_NM'][i] = 'CN'
    elif yhat[i][1] > yhat[i][0] and yhat[i][1] > yhat[i][2]:
        test_labels['DIAG_NM'][i] = 'Dem'
    elif yhat[i][2] > yhat[i][1] and yhat[i][2] > yhat[i][0]:
        test_labels['DIAG_NM'][i] = 'MCI'

In [43]:
yhat[0:10]

array([[9.9923480e-01, 6.6231260e-05, 6.9894904e-04],
       [9.9805570e-01, 2.7420092e-04, 1.6700751e-03],
       [9.9978226e-01, 1.9995385e-04, 1.7750366e-05],
       [9.8011750e-01, 1.5429921e-03, 1.8339552e-02],
       [7.5909758e-01, 5.7234475e-03, 2.3517908e-01],
       [3.1776323e-03, 1.4472807e-04, 9.9667764e-01],
       [9.9855155e-01, 2.0793754e-05, 1.4275986e-03],
       [7.7341706e-01, 1.5467506e-04, 2.2642830e-01],
       [9.9977511e-01, 2.7414979e-05, 1.9750177e-04],
       [9.9974769e-01, 1.9074694e-05, 2.3319699e-04]], dtype=float32)

In [44]:
inv_s2 = le.inverse_transform(yhat).ravel()

In [45]:
inv_s2

array(['CN', 'CN', 'CN', 'CN', 'CN', 'MCI', 'CN', 'CN', 'CN', 'CN', 'MCI',
       'CN', 'MCI', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN',
       'MCI', 'CN', 'CN', 'MCI', 'CN', 'CN', 'CN', 'CN', 'MCI', 'CN',
       'MCI', 'CN', 'CN', 'CN', 'CN', 'MCI', 'CN', 'Dem', 'MCI', 'CN',
       'MCI', 'CN', 'CN', 'CN', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'MCI',
       'MCI', 'CN', 'MCI', 'MCI', 'CN', 'CN', 'MCI', 'CN', 'Dem', 'CN',
       'MCI', 'MCI', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'MCI', 'CN',
       'MCI', 'CN', 'CN', 'MCI', 'CN', 'MCI', 'CN', 'CN', 'CN', 'CN',
       'CN', 'CN', 'CN', 'CN', 'CN', 'MCI', 'CN', 'CN', 'CN', 'CN', 'MCI',
       'CN', 'MCI', 'CN', 'CN', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'MCI',
       'CN', 'MCI', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'Dem',
       'CN', 'MCI', 'CN', 'MCI', 'CN', 'MCI', 'CN', 'CN', 'CN', 'CN',
       'CN', 'CN', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'Dem', 'CN', 'CN',
       'MCI', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'MCI', 'CN', 'MCI', 'CN'

In [46]:
test_labels.to_csv('drive/MyDrive/content/predict18.csv', index=False)

In [47]:
import seaborn as sns
import matplotlib.pyplot as plt